# **VGGNet Paper Implementation**

p*   Paper link : https://arxiv.org/pdf/1409.1556.pdf

In [38]:
import torch
import torch.nn as nn

In [39]:
cfg = [64, 64, 'M', 
       128, 128, 'M', 
       256, 256, 256, 'M',
       512, 512, 512, 'M', 
       512, 512, 512, 'M']

In [40]:
class VGGNet(nn.Module):
  def __init__(self, model, num_classes=10, init_weights=True):
    super(VGGNet, self).__init__()  

    self.conv = creat_conv(model)

    self.fc = nn.Sequential(
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, num_classes),
    )

    if init_weights:
      self._init_weights()

  def forward(self, x):
    x = self.conv(x)
    x = x.view(-1, 512 * 7 * 7)
    x = self.fc(x)
    return x

  def _init_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

def creat_conv(cfg):
  layers = []
  in_channels = 3

  for v in cfg:
    if v == 'M':
      layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
    else:
      conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
      layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
  in_channels = v
  return nn.Sequential(*layers)